## Stream video data to the NN

In [ ]:
%load_ext autoreload
%autoreload 2

#import rospy
import time
import os
import tensorflow as tf
from deep_car.model import get_model
from deep_car.data import get_steering_delta, get_steering_hist
#from sensor_msgs.msg import CompressedImage
#from std_msgs.msg import String,Int16


check = 1
history = []



In [ ]:
def callbackNN(img):
    pred_steering = nn.getPrediciton(img, history)
    pub_steering.publish(pred_steering)
    
    
rospy.init_node('PixelDriveNN')

In [ ]:
sub = rospy.topics.Subscriber("/app/camera/rgb/image_raw/compressed", CompressedImage, callbackNN)

In [ ]:
pub_steering = rospy.Publisher("/manual_control/steering", Int16, queue_size=10)
pub_speed = rospy.Publisher("/manual_control/speed", Int16, queue_size=10)

In [ ]:
pub_steering.publish(0)
time.sleep(0.5)
for i in range(90):
    time.sleep(0.1)
    pub_steering.publish(i)

In [ ]:
pub_speed.publish(0)

In [ ]:
gpu_options = tf.GPUOptions(allow_growth=True) 
sess = tf.InteractiveSession(config=tf.ConfigProto(gpu_options=gpu_options))


len_steering_hist = 4 #get_steering_hist(batch).shape[-1]
len_steering_delta = 6 #get_steering_delta(batch).shape[-1]
y_delta_buckets = 9
input_size = (64,48)
crop_size = input_size
x = tf.placeholder(tf.float32, shape=[None, crop_size[1], crop_size[0], 1], name='image')
steering_hist = tf.placeholder(tf.float32, shape=[None, len_steering_hist], name='steering_hist')

y_abs_true, y_delta_true, opt_op, \
    y_abs_discr_prob, y_abs_loss, y_delta_prob, y_delta_loss = \
        get_model(x, steering_hist, len_steering_delta, y_delta_buckets)
        
        
saver = tf.train.import_meta_graph('../data/model/steering_mixture_prob_exp.ckpt.meta')
saver.restore(sess, tf.train.latest_checkpoint('../data/model/'))

In [ ]:
saver.restore(sess, tf.train.latest_checkpoint('../data/model/'))